## Task 1 – Import required libraries


### Here, we are importing the following required libraries:
* csv for writing data to a CSV file
* pandas for creating the data elements
* selenium for sending HTTP requests to the website
* BeautifulSoup for parsing the HTML source code of the webpage
* Webdriver_manager for automatated installation and management of web drivers
* time for introducing a delay in our program.

In [26]:
import time
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Task 2 – Setup the Selenium Webdriver


In [28]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


## Task 3 – Generate the functions
* Open Indeed in a web browser
* Search for jobs based on user input
* Scrape and Parse Job postings
* Save data into a CSV


In [30]:
# Open Indeed and Search for Jobs
def search_jobs(job_title, location):
    url = "https://ca.indeed.com/"
    driver.get(url)
    time.sleep(2)  # Allow time for page to load

    # Find search fields
    job_search = driver.find_element(By.ID, "text-input-what")
    location_search = driver.find_element(By.ID, "text-input-where")

    # Clear and input job title & location
    job_search.clear()
    job_search.send_keys(job_title)

    location_search.clear()
    location_search.send_keys(location)

    # Press Enter to search
    job_search.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow results to load


# Parse Job Listings
def scrape_jobs():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    job_cards = soup.find_all("div", class_="job_seen_beacon")

    jobs = []

    for job in job_cards:
        title_elem = job.find("h2", class_="jobTitle css-1psdjh5 eu4oa1w0")
        company_elem = job.find("span", class_="css-1h7lukg eu4oa1w0")
        location_elem = job.find("div", class_="css-1restlb eu4oa1w0")
        link_elem = job.find("a", class_="jcs-JobTitle css-1baag51 eu4oa1w0")

        if title_elem and company_elem and location_elem and link_elem:
            job_title = title_elem.text.strip()
            company = company_elem.text.strip()
            location = location_elem.text.strip()
            job_link = "https://ca.indeed.com" + link_elem["href"]

            jobs.append([job_title, company, location, job_link])

    return jobs


# Save Data to CSV
def save_to_csv(data, filename="indeed_jobs.csv"):
    df = pd.DataFrame(data, columns=["Job Title", "Company", "Location", "Job Link"])
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")


## Task 4 – Define the main function.


In [32]:
# Run the script
if __name__ == "__main__":
    job_title = input("Enter job title to search: ")
    location = input("Enter job location (leave blank for Etobicoke as Default): ")  # No default location

    search_jobs(job_title, location)
    jobs_data = scrape_jobs()

    if jobs_data:
        save_to_csv(jobs_data)
    else:
        print("No jobs found.")

    driver.quit()

Enter job title to search:  Data Analyst
Enter job location (leave blank for Etobicoke as Default):  


Data saved to indeed_jobs.csv


In [49]:
from tabulate import tabulate

# Read the CSV file
df = pd.read_csv('indeed_jobs.csv')

# Print the dataframe as a table
print(tabulate(df, headers='keys'))


    Job Title                                       Company                                Location                             Job Link
--  ----------------------------------------------  -------------------------------------  -----------------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------